# Orbits of planets and new spacecraft
# including Parker Solar Probe, JUNO, BepiColombo, and Solar Orbiter
***

*EUHFORIA* requires data of the orbits of spacecraft/planets in order to be able to extract data at the spacecraft position from the simulation. This notebook shows the essential steps of how the spacecraft position information is stored and accessed by *EUHFORIA*.

## 1. Getting orbit data

The first step is to get data of the orbit of the spacecraft/planet etc. that one is interested in. In particular, what is needed is the position of the object in HEEQ coordinates as a function of time. For most objects, so called SPICE kernels are provided by NASA/ESA etc. We will use these together in the context of the [SpiceyPy](https://github.com/AndrewAnnex/SpiceyPy) package to compute the orbit data. Another usefull resource is the
[NASA HelioWeb](https://omniweb.gsfc.nasa.gov/coho/helios/heli.html) tool.

### Downloading basic SPICE kernels

Planet ephemeris kernel file:

In [1]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de430.bsp -P spice_kernels/

--2020-03-05 18:37:52--  https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de430.bsp
Risoluzione di naif.jpl.nasa.gov (naif.jpl.nasa.gov)... 137.78.232.95
Connessione a naif.jpl.nasa.gov (naif.jpl.nasa.gov)|137.78.232.95|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 119741440 (114M) [text/plain]
Salvataggio in: "spice_kernels/de430.bsp.1"

de430.bsp.1         100%[===================>] 114.19M  5.42MB/s    in 33s     

2020-03-05 18:38:31 (3.44 MB/s) - "spice_kernels/de430.bsp.1" salvato [119741440/119741440]



Orientation data:

In [2]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/pck00010.tpc -P spice_kernels/

--2020-03-05 18:38:31--  https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/pck00010.tpc
Risoluzione di naif.jpl.nasa.gov (naif.jpl.nasa.gov)... 137.78.232.95
Connessione a naif.jpl.nasa.gov (naif.jpl.nasa.gov)|137.78.232.95|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 126143 (123K) [text/plain]
Salvataggio in: "spice_kernels/pck00010.tpc.1"

pck00010.tpc.1      100%[===================>] 123.19K   158KB/s    in 0.8s    

2020-03-05 18:38:32 (158 KB/s) - "spice_kernels/pck00010.tpc.1" salvato [126143/126143]



Leap second data:

In [3]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls -P spice_kernels/

--2020-03-05 18:38:33--  https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls
Risoluzione di naif.jpl.nasa.gov (naif.jpl.nasa.gov)... 137.78.232.95
Connessione a naif.jpl.nasa.gov (naif.jpl.nasa.gov)|137.78.232.95|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 5257 (5.1K) [text/plain]
Salvataggio in: "spice_kernels/naif0012.tls.1"

naif0012.tls.1      100%[===================>]   5.13K  --.-KB/s    in 0s      

2020-03-05 18:38:33 (85.0 MB/s) - "spice_kernels/naif0012.tls.1" salvato [5257/5257]



Heliospheric and geomagnetic coordinate system definitions:

In [4]:
!wget http://proba2.sidc.be/aux/data/spice/kernels/fk/heliospheric.tf -P spice_kernels/

--2020-03-05 18:38:34--  http://proba2.sidc.be/aux/data/spice/kernels/fk/heliospheric.tf
Risoluzione di proba2.sidc.be (proba2.sidc.be)... 193.190.230.16
Connessione a proba2.sidc.be (proba2.sidc.be)|193.190.230.16|:80... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 9767 (9.5K)
Salvataggio in: "spice_kernels/heliospheric.tf.3"

heliospheric.tf.3   100%[===================>]   9.54K  --.-KB/s    in 0.001s  

2020-03-05 18:38:34 (11.1 MB/s) - "spice_kernels/heliospheric.tf.3" salvato [9767/9767]



In [5]:
!wget http://proba2.sidc.be/aux/data/spice/kernels/fk/geomagnetic.tf -P spice_kernels/

--2020-03-05 18:38:34--  http://proba2.sidc.be/aux/data/spice/kernels/fk/geomagnetic.tf
Risoluzione di proba2.sidc.be (proba2.sidc.be)... 193.190.230.16
Connessione a proba2.sidc.be (proba2.sidc.be)|193.190.230.16|:80... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 2183 (2.1K)
Salvataggio in: "spice_kernels/geomagnetic.tf.3"

geomagnetic.tf.3    100%[===================>]   2.13K  --.-KB/s    in 0s      

2020-03-05 18:38:34 (122 MB/s) - "spice_kernels/geomagnetic.tf.3" salvato [2183/2183]



Earth data:

In [6]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_720101_070426.bpc -P spice_kernels/

--2020-03-05 18:38:34--  https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_720101_070426.bpc
Risoluzione di naif.jpl.nasa.gov (naif.jpl.nasa.gov)... 137.78.232.95
Connessione a naif.jpl.nasa.gov (naif.jpl.nasa.gov)|137.78.232.95|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 8603648 (8.2M) [text/plain]
Salvataggio in: "spice_kernels/earth_720101_070426.bpc.1"

earth_720101_070426 100%[===================>]   8.21M  1.47MB/s    in 9.3s    

2020-03-05 18:38:44 (903 KB/s) - "spice_kernels/earth_720101_070426.bpc.1" salvato [8603648/8603648]



In [7]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_070425_370426_predict.bpc -P spice_kernels/

--2020-03-05 18:38:44--  https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_070425_370426_predict.bpc
Risoluzione di naif.jpl.nasa.gov (naif.jpl.nasa.gov)... 137.78.232.95
Connessione a naif.jpl.nasa.gov (naif.jpl.nasa.gov)|137.78.232.95|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 5751808 (5.5M) [text/plain]
Salvataggio in: "spice_kernels/earth_070425_370426_predict.bpc.1"

earth_070425_370426 100%[===================>]   5.49M  1.15MB/s    in 7.8s    

2020-03-05 18:38:53 (719 KB/s) - "spice_kernels/earth_070425_370426_predict.bpc.1" salvato [5751808/5751808]



### Creating the SPICE meta file

After the kernels have been downloaded, a meta-file for loading them needs to be created:

In [8]:
%%writefile spice_kernels/meta.txt

\begindata
KERNELS_TO_LOAD=(
 'spice_kernels/de430.bsp',
 'spice_kernels/pck00010.tpc',
 'spice_kernels/naif0012.tls',
 'spice_kernels/heliospheric.tf',
 'spice_kernels/geomagnetic.tf',
 'spice_kernels/earth_070425_370426_predict.bpc',
 'spice_kernels/earth_720101_070426.bpc')
\begintext

Overwriting spice_kernels/meta.txt


## 2. Computing the orbits

After the needed kernels have been downloaded, we can start using them to compute the orbits using SpiceyPy. 

In [9]:
import spiceypy as spice
import datetime
import numpy as np

In [10]:
astronomical_unit  = 149597870700.0

Load the kernels:

In [11]:
spice.furnsh("spice_kernels/meta.txt")

Define function to compute HEEQ coordinates of an object:

In [12]:
def get_heeq(body, dates):
    """Computes the HEEQ coordinates for the given body at the given times.
    
    Args:
        body (str) : Name of the body
        dates      : List of datetime objects for which to computes the position
    Returns:
        HEEQ tuple of r [m], lat [rad], lon [rad] 
    """
    
    num_dates = len(dates)
        
    r     = np.zeros(num_dates)
    lon   = np.zeros(num_dates)
    lat   = np.zeros(num_dates)
    
    for idx, date in enumerate(dates):
    
        # Compute epoch time
        et = spice.str2et(datetime.datetime.strftime(date, format="%Y-%m-%dT%H:%M:%S"))

        # Compute position
        pos_and_vel, light_travel_time = spice.spkezr(body, et, 'HEEQ', 'None', 'Sun')
        
        # Position in spherical coordinates
        heeq_r_km, lon[idx], lat[idx] = spice.reclat(pos_and_vel[0:3])

        # Convert unit to m
        r[idx] = heeq_r_km*1e3
                
    return r, lat, lon

Define the dates of interest:

In [13]:
dates = [datetime.datetime(1975, 1, 1, 12, 0) + datetime.timedelta(days=x) for x in range(0, 21915)]
# 1 January 1975 + 21915 days --> 1 January 2035

Let's compute and save the position for a selection of objects for the given dates. Note that I here use the barycenter position simply to avoid downloading further kernels.

In [14]:
for obj in ('Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto'):
    
    # Get HEEQ position
    r, lat, lon = get_heeq(obj + ' barycenter', dates)
    
    # Save to file
    np.savez("orbit_data/"+obj.lower(), dates=dates, r=r, lat=lat, lon=lon)

# Parker Solar Probe

PSP kernels are located at https://sppgway.jhuapl.edu/ancil_products

Note: kernels from the link above does not work so we are using a .bsp file from Bodgan Nicula (nominal orbit)

### Load and save data for PSP

Using nominal ephemeris file from Bogdan first (has to be already stored locally)

In [15]:
import glob

In [16]:
# loading nominal ephemeris file
# spp_nom_20180812_20250831_v034_RO1_TCM1.bsp
for files in glob.glob("spice_kernels/spp_nom_20180812_20250831_v034_RO1_TCM1.bsp"):
    spice.furnsh(files)

In [17]:
# from 13 Aug 2018 until 31 Aug 2025
dates = [datetime.datetime(2018, 8, 13, 0, 0) + datetime.timedelta(days=x) for x in range(0, 2569)]

In [18]:
r, lat, lon = get_heeq('SPP', dates)

In [19]:
np.savez("orbit_data/psp_R01", dates=dates, r=r, lat=lat, lon=lon)

Let's try downloading nominal ephemeris files from the server (commands fails for some reason, so I have already downloaded the files in /spice_kernels/ manually)

In [20]:
!wget -r --no-parent -nH -nd -A 'spp_nom_20180812_20250831_v035_RO2.bsp' https://sppgway.jhuapl.edu/lpredict_ephem/ -P spice_kernels/

--2020-03-05 17:34:20--  https://sppgway.jhuapl.edu/lpredict_ephem/
Risoluzione di sppgway.jhuapl.edu (sppgway.jhuapl.edu)... 128.244.181.218
Connessione a sppgway.jhuapl.edu (sppgway.jhuapl.edu)|128.244.181.218|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 316 [text/html]
Salvataggio in: "spice_kernels/index.html.tmp"

index.html.tmp      100%[===================>]     316  --.-KB/s    in 0s      

2020-03-05 17:34:21 (4.24 MB/s) - "spice_kernels/index.html.tmp" salvato [316/316]

Rimozione di spice_kernels/index.html.tmp poich'e deve essere rifiutato.

TERMINATO --2020-03-05 17:34:21--
Tempo totale: 0.5s
Scaricati: 1 file, 316 in 0s (4.24 MB/s)


In [21]:
# loading nominal ephemeris file
for files in glob.glob("spice_kernels/spp_nom_*_RO2.bsp"):
    spice.furnsh(files)

In [22]:
# from 13 Aug 2018 until 31 Aug 2025
dates = [datetime.datetime(2018, 8, 13, 0, 0) + datetime.timedelta(days=x) for x in range(0, 2575)]

In [23]:
r, lat, lon = get_heeq('SPP', dates)

In [24]:
np.savez("orbit_data/psp_R02", dates=dates, r=r, lat=lat, lon=lon)

Finally, let's download the reconstructed ephemeris files from the server

In [25]:
!wget -r --no-parent -nH -nd -A '201*/spp_recon_*.bsp' https://sppgway.jhuapl.edu/recon_ephem/ -P spice_kernels/

--2020-03-05 17:34:22--  https://sppgway.jhuapl.edu/recon_ephem/
Risoluzione di sppgway.jhuapl.edu (sppgway.jhuapl.edu)... 128.244.181.218
Connessione a sppgway.jhuapl.edu (sppgway.jhuapl.edu)|128.244.181.218|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 316 [text/html]
Salvataggio in: "spice_kernels/index.html.tmp"

index.html.tmp      100%[===================>]     316  --.-KB/s    in 0s      

2020-03-05 17:34:22 (4.19 MB/s) - "spice_kernels/index.html.tmp" salvato [316/316]

Rimozione di spice_kernels/index.html.tmp poich'e deve essere rifiutato.

TERMINATO --2020-03-05 17:34:22--
Tempo totale: 0.5s
Scaricati: 1 file, 316 in 0s (4.19 MB/s)


In [26]:
# loading reconstructed (past) ephemeris file
for files in glob.glob("spice_kernels/spp_recon_*.bsp"):
    spice.furnsh(files)

In [27]:
# from 13 Aug 2018 until 16 Apr 2018
dates = [datetime.datetime(2018, 8, 13, 0, 0) + datetime.timedelta(days=x) for x in range(0, 245)]

In [28]:
r, lat, lon = get_heeq('SPP', dates)

In [29]:
np.savez("orbit_data/psp_recon", dates=dates, r=r, lat=lat, lon=lon)

# JUNO 

JUNO kernels are located at https://naif.jpl.nasa.gov/pub/naif/JUNO/kernels/spk/ 

Download recorded (i.e. past ?) data:

In [30]:
!wget -r --no-parent -nH -nd -A 'spk_rec_*.bsp' https://naif.jpl.nasa.gov/pub/naif/JUNO/kernels/spk/ -P spice_kernels/

--2020-03-05 17:34:23--  https://naif.jpl.nasa.gov/pub/naif/JUNO/kernels/spk/
Risoluzione di naif.jpl.nasa.gov (naif.jpl.nasa.gov)... 137.78.232.95
Connessione a naif.jpl.nasa.gov (naif.jpl.nasa.gov)|137.78.232.95|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 404 Not Found
2020-03-05 17:34:23 ERRORE 404: Not Found.



### Load and save data for JUNO

In [31]:
import glob

In [32]:
# loading recorded (past) ephemeris 
for files in glob.glob("spice_kernels/spk_rec_*.bsp"):
    spice.furnsh(files)

In [33]:
# currently from 6 Aug 2011 until 19 Jun 2019
dates = [datetime.datetime(2011, 8, 6, 0, 0) + datetime.timedelta(days=x) for x in range(0, 2875)]

In [34]:
r, lat, lon = get_heeq('JUNO', dates)

In [35]:
np.savez("orbit_data/juno", dates=dates, r=r, lat=lat, lon=lon)

# BepiColombo 

BepiColombo kernels are located at: ftp://spiftp.esac.esa.int/data/SPICE/BEPICOLOMBO/kernels/

Let us first manually store the .bsp files under local parth: ./spice_kernels/

The good files are named: 'bc_mpo_fpc_NNNNN_YYYYMMDD_YYYYMMDD_vNN.psp'

Once the files are there, we can continue with the next steps

### Load and save data for BepiColombo

In [36]:
import glob

In [37]:
# loading recorded (past) ephemeris 
for files in glob.glob("spice_kernels/bc_mpo_fcp_*_20181020_20251102_v01.bsp"):
    spice.furnsh(files)

In [38]:
# currently from 21 Oct 2018 until 10 Nov 2025
dates = [datetime.datetime(2018, 10, 21, 0, 0) + datetime.timedelta(days=x) for x in range(0, 2569)]

In [39]:
r, lat, lon = get_heeq('MPO', dates)

In [40]:
np.savez("orbit_data/mpo", dates=dates, r=r, lat=lat, lon=lon)

# Solar Orbiter

In this case we assume to already have the good .bsp file saved under ./spice_kernels/

### Load and save data for Solar Orbiter

In [16]:
import glob

In [17]:
# loading recorded (past) ephemeris 
for files in glob.glob("spice_kernels/solo_ANC_soc-orbit_20200207-20300902_V01.bsp"):
    spice.furnsh(files)

In [22]:
# currently from 07 Feb 2020 until 02 Sep 2030
dates = [datetime.datetime(2020, 2, 7, 0, 0) + datetime.timedelta(days=x) for x in range(0, 3855)]

In [23]:
r, lat, lon = get_heeq('SOLAR ORBITER', dates)

In [24]:
np.savez("orbit_data/solo", dates=dates, r=r, lat=lat, lon=lon)